In [31]:
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

# 1. Load collected data

In [32]:
collected_data = pd.read_json('../extractor_de_metadatos/all_collected_data.jsonl', lines=True)
print(f"Columnas: {list(collected_data.columns)}\nNúmero de Filas: {len(collected_data)}")

Columnas: ['community_page', 'document_page', 'breadcrumb', 'file_urls', 'files']
Número de Filas: 26837


# 2. Transform and clean up

In [33]:
collected_data['pdf_path'] = collected_data['files'].str[0].str['path']
collected_data['xml_path'] = collected_data['files'].str[1].str['path']

In [34]:
from parsel import Selector
import codecs

def read_file(filename):
    with codecs.open(filename, 'r', 'utf-8') as f:
        return f.read()
    
def get_field(xml_string: str, attribute, value, return_all_matches=False):
    sel = Selector(text=xml_string)
    sel.register_namespace('dim', 'http://www.dspace.org/xmlns/dspace/dim')
    matches = sel.xpath(f'//dim/field[@{attribute}="{value}"]/text()')
    if return_all_matches:
        return matches.getall()
    return matches.get()

def get_title(xml_string):
    return  get_field(xml_string, 'element', 'title')

def get_abstract(xml_string):
    return get_field(xml_string, 'qualifier', 'abstract')

def get_subjects(xml_string):
    subjects_extracted = get_field(xml_string, 'element', 'subject', return_all_matches=True)
    return [s.upper() for s in subjects_extracted]

collected_data['xml_content'] = collected_data['xml_path'].progress_map(lambda xml_path: read_file('../scrapy/files/' + xml_path), na_action='ignore')

# get these fields as arrays 
collected_data['title'] = collected_data['xml_content'].progress_map(get_title, na_action='ignore')
collected_data['abstract'] = collected_data['xml_content'].progress_map(get_abstract, na_action='ignore')
collected_data['subjects'] = collected_data['xml_content'].progress_map(get_subjects, na_action='ignore')
collected_data.sample(10)

100%|█████████▉| 26826/26837 [00:04<00:00, 5461.76it/s]


,community_page,document_page,breadcrumb,file_urls,files,pdf_path,xml_path,xml_content,title,abstract,subjects
11889,https://repositorio.umsa.bo/handle/123456789/3...,https://repositorio.umsa.bo/handle/123456789/8986,"[DSpace Home, Facultad de Tecnología, Carrera ...",[https://repositorio.umsa.bo/bitstream/handle/...,[{'url': 'https://repositorio.umsa.bo/bitstrea...,full/31b9effbd47515f49ce88c3f3a859df576a30dc4,full/a385db4d303b07dda14bfa3876febd2c534c7075.xml,"<?xml version=""1.0"" encoding=""UTF-8""?><mets:ME...",Diseño de una camara de secado de pintura de a...,El presente proyecto tiene como objetivo funda...,"[CAMARA DE SECADO, SECADO DE PINTURA, PINTURA ..."
3540,https://repositorio.umsa.bo/handle/123456789/3...,https://repositorio.umsa.bo/handle/123456789/2...,"[DSpace Home, Facultad de Humanidades y Cienci...",[https://repositorio.umsa.bo/bitstream/handle/...,[{'url': 'https://repositorio.umsa.bo/bitstrea...,full/7ac957e712c18efdd96874ff038a9ad357284700,full/fc143c0a262e95066e8c53f0414439f9063f9f4c.xml,"<?xml version=""1.0"" encoding=""UTF-8""?><mets:ME...",Los mineros de Larecaja : la explotación del o...,1. La explotacion aurifera en el distrito de L...,"[HISTORIA, HISTORIA - LARECAJA, EXPLOTACIONES ..."
22015,https://repositorio.umsa.bo/handle/123456789/3...,https://repositorio.umsa.bo/handle/123456789/6072,"[DSpace Home, Facultad de Agronomía, Carrera d...",[https://repositorio.umsa.bo/bitstream/handle/...,[{'url': 'https://repositorio.umsa.bo/bitstrea...,full/20393dc91605e20e5407eae72dc36de24ffea1f1,full/6c59f4abaa1bdc4af2ff60dad10a8e9f6eb11b8f.xml,"<?xml version=""1.0"" encoding=""UTF-8""?><mets:ME...","Caracterización participativa sobre usos, rest...","La situación de la cañahua, en el altiplano No...","[CAÑAHUA, CHENOPODIUM PALLIDICAULE AELLEN, CON..."
6767,https://repositorio.umsa.bo/handle/123456789/3...,https://repositorio.umsa.bo/handle/123456789/2...,"[DSpace Home, Facultad de Derecho y Ciencias P...",[https://repositorio.umsa.bo/bitstream/handle/...,[{'url': 'https://repositorio.umsa.bo/bitstrea...,full/5fa0134df7d5b10ba73b5a162eb1da304e696cd1,full/a38d20bc2dc8bb0f443498d8fe1d0766f7cf2d3f.xml,"<?xml version=""1.0"" encoding=""UTF-8""?><mets:ME...",Estado de situación de la normativa referida a...,El objetivo es poner en claro y conocimiento a...,"[BRIGADAS DE PROTECCION A LA FAMILIA, VIOLENCI..."
10754,https://repositorio.umsa.bo/handle/123456789/3...,https://repositorio.umsa.bo/handle/123456789/7769,"[DSpace Home, Facultad de Ciencias Puras y Nat...",[https://repositorio.umsa.bo/bitstream/handle/...,[{'url': 'https://repositorio.umsa.bo/bitstrea...,full/ab55efd7853d8061fe041110914be290e5323df7,full/faf7755af1cfab042262dc6827ec1198ea7f9424.xml,"<?xml version=""1.0"" encoding=""UTF-8""?><mets:ME...",Tutor Inteligente para la enseñanza del proces...,En la actualidad existen estudiantes universit...,"[TUTOR INTELIGENTE, ENSEÑANZA DEL PROCESO ENFE..."
25592,https://repositorio.umsa.bo/handle/123456789/1...,https://repositorio.umsa.bo/handle/123456789/9572,"[DSpace Home, Área Vicerrectorado, Biblioteca ...",[https://repositorio.umsa.bo/bitstream/handle/...,[{'url': 'https://repositorio.umsa.bo/bitstrea...,full/f069ba60a77a174a658a1127a4873299284ea841,full/4a5ae2eeb34229ec639709c0fb3bf63e23ee5d98.xml,"<?xml version=""1.0"" encoding=""UTF-8""?><mets:ME...",Homenaje de la Sociedad Geográfico-Histórica d...,Boletín Nº1 de la Sociedad Geográfica Históric...,[SOCIEDAD GEOGRAFICO HISTORICA DE POTOSI CREAC...
3162,https://repositorio.umsa.bo/handle/123456789/3...,https://repositorio.umsa.bo/handle/123456789/3...,"[DSpace Home, Facultad de Medicina, Enfermería...",[https://repositorio.umsa.bo/bitstream/handle/...,[{'url': 'https://repositorio.umsa.bo/bitstrea...,full/59922a133cb5247887a783565896aa8a9e7863ac,full/6c433a04f47945b030876220a4c55bd7b3cd99fd.xml,"<?xml version=""1.0"" encoding=""UTF-8""?><mets:ME...",Conocimiento de la lista de verificación de ci...,La lista de verificación de cirugía segura de ...,"[ENFERM

In [35]:
# for reporting purposes

collected_data[['title', 'abstract', 'subjects']]

,title,abstract,subjects
0,Cálculo de las entradas ordinarias relativas a...,Relación desglosada de los ingresos del Presu...,[PRESUPUESTO NACIONAL 1931]
1,Defensa del general José C. Quiroz en la acusa...,Defensa en los estrados judiciales del Gral. J...,"[JUICIO DE RESPONSABILIDAD, CONTRATOS VICKERS]"
2,Cálculo de las entradas ordinarias y extraordi...,Cálculo de las entradas ordinarias clasificada...,[PRESUPUESTOS DEPARTAMENTALES 1929]
3,Optimización del sistema de seguridad y etique...,Nuestro país se encuentra en vías de crecimien...,"[SISTEMA DE SEGURIDAD, TRANSMISIÓN DE DATOS, S..."
4,Desarrollo de un sistema electrónico que permi...,Este documento trata de la manera de desarroll...,"[DETECTOR DE INCENDIOS, EMPRESA ROSVANIA, PREV..."
...,...,...,...
26832,Alerta policial para enfrentar la violencia es...,La presente propuesta de intervención socioedu...,"[EDUCACION ESCOLAR, POLICIA, VIOLENCIA, UNIDAD..."
26833,Solidaridad básica y solidaridad limitada:,La cuestión de la minería en Bolivia ha sido i...,"[COOPERATIVAS MINERAS, SOLIDARIDAD BÁSICA, SOL..."
26834,Examen de confiabilidad de los registros y est...,El Servicio Nacional de Aerofotogrametría es u...,"[CONFIABILIDAD DE REGISTROS, AUDITORIA GUBERNA..."
26835,Auditoría de confiabilidad de registros y esta...,En Cumplimiento al Programa Operativo Anual de...,"[AUDITORIA DE CONFIABILIDAD, CIENCIAS GEOLÓGIC..."


In [5]:
collected_data.dropna(subset=['title', 'abstract', 'subjects'], inplace=True)
print(len(collected_data))

26044


In [6]:
min_subjects_mask = collected_data['subjects'].str.len() >= collected_data['subjects'].str.len().mean() - 0.5 * collected_data['subjects'].str.len().std()
max_subjects_mask = collected_data['subjects'].str.len() <= collected_data['subjects'].str.len().mean() + 2 * collected_data['subjects'].str.len().std()
subjects_range_mask = min_subjects_mask & max_subjects_mask
collected_data = collected_data[subjects_range_mask]
len(collected_data)

18573

In [7]:
min_abstract_mask = collected_data['abstract'].str.len() >= 100
max_abstract_mask = collected_data['abstract'].str.len() <= collected_data['abstract'].str.len().mean() + 1 * collected_data['abstract'].str.len().std()
abstract_range_mask = min_abstract_mask & max_abstract_mask
collected_data = collected_data[abstract_range_mask]
len(collected_data)

16479

In [8]:
min_title_mask = collected_data['title'].str.len() >= 50
max_title_mask = collected_data['title'].str.len() <= collected_data['title'].str.len().mean() + 1 * collected_data['title'].str.len().std()
title_range_mask = min_title_mask & max_title_mask
collected_data = collected_data[max_title_mask]
len(collected_data)

13931

In [9]:
# delete this cell
from unidecode import unidecode

def keep_exact_matches(row):
    filtered_subjects = [] 
    title = unidecode(row['title']).lower()
    abstract = unidecode(row['abstract']).lower()

    for s in row['subjects']:
        subject = unidecode(s).lower()
        index_in_title = title.find(subject)
        if index_in_title != -1:
            # print(s, '->', row['title'][index_in_title: index_in_title + len(subject)])
            filtered_subjects.append(row['title'][index_in_title: index_in_title + len(subject)])
            continue
        index_in_abstract = abstract.find(subject)
        if index_in_abstract != -1:
            # print(s, '->', row['abstract'][index_in_abstract: index_in_abstract + len(subject)])
            filtered_subjects.append(row['abstract'][index_in_abstract: index_in_abstract + len(subject)])
    if len(filtered_subjects) >= 2:
        return filtered_subjects
    else:
        return None

collected_data["subjects"] = collected_data.apply(keep_exact_matches, axis=1)

In [10]:
collected_data.dropna(subset="subjects", inplace=True)
len(collected_data)

8226

In [30]:
collected_data[['title', 'abstract', 'subjects']]

,title,abstract,subjects
3,Optimización del sistema de seguridad y etique...,Nuestro país se encuentra en vías de crecimien...,"[sistema de seguridad, transmisión de datos, c..."
4,Desarrollo de un sistema electrónico que permi...,Este documento trata de la manera de desarroll...,"[Empresa ROSVANIA, prevención de incendios]"
5,Diseño e implementación de un sistema de medic...,El presente proyecto surge con la necesidad de...,"[lactantes mayores, Hospital La Paz, señales e..."
7,Informe de mantenimiento industrial de equipo ...,En el presente informe detallo los trabajos de...,"[mantenimiento industrial, mantenimiento preve..."
9,"Rediseño del sistema de puesta a tierra, para ...",Este proyecto está predispuesto a dar solución...,"[puesta a tierra, materiales eléctricos, insta..."
...,...,...,...
26827,Auditoría operacional al proceso de provisión ...,La presente Auditoría Operacional al Proceso d...,"[Auditoría operacional, medicamentos]"
26830,Promoviendo y fortaleciendo la participación d...,El Centro Integral NAT’s Sagrados Corazones tr...,"[educación, familia, rendimiento escolar]"
26832,Alerta policial para enfrentar la violencia es...,La presente propuesta de intervención socioedu...,"[policia, violencia, Unidades Educativas]"
26833,Solidaridad básica y solidaridad limitada:,La cuestión de la minería en Bolivia ha sido i...,"[Solidaridad básica, solidaridad limitada, Bol..."


In [ ]:
collected_data.to_pickle('collected_data.pkl')